# Cutting Stock Problem

Roll No : 231140028 (Vivek Radadiya)

In [8]:
import cplex
from cplex.exceptions import CplexError
import numpy as np
import math
import time

Data Read

In [9]:
def read_data_from_file(filename):
    
    order_length = []
    demand = []

    with open(filename, 'r') as file:
        stock_length = float(file.readline().split(":")[1])
        num_orders = int(file.readline().split(":")[1])

        # Skip the next line
        file.readline()

        for _ in range(num_orders):
            line = file.readline().split()
            order_length.append(float(line[0]))
            demand.append(int(line[1]))

    return stock_length, num_orders, order_length, demand

Initial Solution Genration

In [10]:
def initial_solution(stock_length, num_orders, order_length, demand):
    # Add your logic for initializing the solution here
    
    intial_sol=[]
    for i in range(num_orders):
        temp=[0]*num_orders
        temp[i]=int(stock_length/order_length[i])
        intial_sol.append(temp)
        
    return intial_sol

variables.add(obj, lb, ub, names, types, columns):

    model.linear_constraints.add(lin_expr,senses,rhs,names)

Master Problem

In [11]:
def create_master_problem(num_orders, demand, initial_sol):
    # Create and configure CPLEX object for the master problem
    master = cplex.Cplex()
    master.set_problem_type(cplex.Cplex.problem_type.LP)
    master.objective.set_sense(master.objective.sense.minimize)

    # Add variables
    variable_objectives = [1] * len(initial_sol)  # Objective coefficients
    variable_lb=[0] * len(initial_sol)
    variable_ub = [cplex.infinity] * len(initial_sol)  # No upper bounds
    variable_names = ['x{}'.format(i) for i in range(len(initial_sol))]
    variable_types = ['I'] * len(initial_sol)  # Integer variables
    
    

    master.variables.add(obj=variable_objectives,lb=variable_lb,ub=variable_ub, names=variable_names)

    for j in range(num_orders):
        constraint_variables = ['x{}'.format(i) for i in range(len(initial_sol))]
        constraint_coefficients = [initial_sol[i][j] for i in range(len(initial_sol))]
        constraint_names = ['c{}'.format(j)]

        master.linear_constraints.add(lin_expr=[[constraint_variables, constraint_coefficients]],senses=['G'],rhs=[demand[j]],names=constraint_names)
        

    return master

Sub Problem (Minimize Zj-Cj)

In [12]:
def solve_pricing(Order_length,demand, dual_var, stock_length):
    I = len(dual_var)
    
     # Create and configure CPLEX object for the master problem
    master2 = cplex.Cplex()
    #master.set_problem_type(cplex.Cplex.problem_type.LP)
    master2.objective.set_sense(master2.objective.sense.maximize)

    # Add variables
    variable_objectives = [i for i in dual_var]  # Objective coefficients
    variable_lb=[0] * len(dual_var)
    variable_ub = [cplex.infinity] * len(dual_var)  # No upper bounds
    variable_names = ['y{}'.format(i) for i in range(len(dual_var))]
    variable_types = ['I'] * len(dual_var)  # Integer variables
    
    

    master2.variables.add(obj=variable_objectives,lb=variable_lb,ub=variable_ub, names=variable_names, types=variable_types)

    # Add variable for the subproblem
    
    # Add constraint for the subproblem
    constraint_variables = ['y{}'.format(i) for i in range(len(dual_var))]
    constraint_coefficients = [Order_length[i] for i in range(len(dual_var))]
    master2.linear_constraints.add(lin_expr=[[constraint_variables, constraint_coefficients]],senses=['L'],rhs=[stock_length])

    # Solve the subproblem
    return master2
    

Logging the Pattern Genrated

In [13]:
def solution_logging(stock_length, num_orders, Order_length, demand):

    file_path = "log_file.txt"

    with open(file_path, 'a') as file:
    
        initial_sol = initial_solution(stock_length, num_orders, Order_length, demand)
        for i in initial_sol:
            file.write(f"{i}\n")

        j=0    
        while(True):
            j=j+1   
            master1 = create_master_problem(num_orders, demand, initial_sol)
            master1.solve()
            no_sheets=master1.solution.get_objective_value()
            curr_sol=master1.solution.get_values()
            dual_var=master1.solution.get_dual_values()
            master2=solve_pricing(Order_length,demand, dual_var, stock_length)
            master2.solve()
            new_pattern=master2.solution.get_values()
            file.write(f"\n \n Iteration No.   : {j}\n")
            file.write(f"No. of sheets to be cut: {no_sheets}\n")
            file.write(f"Current solution: {curr_sol}\n")
            file.write(f"Dual Values :{dual_var}\n")
            file.write(f"New Pattern Genrated is: {new_pattern}\n")

            if( new_pattern in initial_sol ):
                break
            else:
                initial_sol.append((new_pattern))
    return initial_sol,master1

Main Fucntion

In [14]:

filename = "D:\\CPLEX\\ATT-Projects\\input.txt"
log_path="D:\CPLEX\ATT-Projects\Output.txt"
stock_length, num_orders, Order_length, demand = read_data_from_file(filename)

start_time=time.time()
Patterns, final_sol=solution_logging(stock_length, num_orders, Order_length, demand)
file_path = "output.txt"
end_time=time.time()

sheets=math.ceil(final_sol.solution.get_objective_value())
Sol_cut=final_sol.solution.get_values()
sheets=sum([math.ceil(i) for i in Sol_cut])
total_m_required=sheets*stock_length
total_m_cut=0
for i in range(6):
    for j in range(len(Sol_cut)):
        total_m_cut+=Order_length[i]*Patterns[j][i]*(math.ceil(Sol_cut[j]))

per_waste=(1-(total_m_cut/total_m_required))*100
with open(file_path, 'a') as file:
    file.write(f"\n No. Of Total Sheets Required is : {sheets}\n")
    file.write(f"\n Length Used is : {total_m_required}\n")
    file.write(f"\n Length Utilized is : {total_m_cut}\n")
    file.write(f"\n % Waste is : {per_waste}\n")
    file.write(f"\n Time taken is : {end_time-start_time}\n")
    for i in range(len(Patterns)):
        file.write(f"\n {Patterns[i]} :     {math.ceil(Sol_cut[i])} \n")

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 6 rows and 6 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.00 ticks)
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve modified 1 coefficients.
Reduced MIP has 1 rows, 6 columns, and 6 nonzeros.
Reduced MIP has 0 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 1 rows, 6 columns, and 6 nonzeros.
Reduced MIP has 0 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.00

  Real time             =    0.11 sec. (0.06 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.11 sec. (0.06 ticks)
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.01 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =           138.333333
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve modified 1 coefficients.
Reduced MIP has 1 rows, 6 columns, and 6 nonzeros.
Reduced MIP has 0 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.01 ticks)
Tried aggregator 1 time

Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.00 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000        5.3406              --- 
      0     0        1.0259     1        0.0000        1.0259        1     --- 
*     0+    0                            1.0000        1.0259             2.59%
      0     0        1.0233     1        1.0000    MIRcuts: 1        2    2.33%
      0     0        1.0219     2        1.0000    MIRcuts: 1        3    2.19%
      0     0        1.0180     2        1.0000    MIRcuts: 1        5    1.80%
      0     0        1.0174     3        1.0000    MIRcuts: 1        6    1.74%
      0     0        1.0165     4        1.0000   ZeroHalf: 1        8    1.65%
      0     0        1.0164     5        1.0000   ZeroHalf: 1        9    1.64%
      0     0        cutoff    